In [2]:
import tensorflow as tf
import numpy as np


In [3]:
import re
import random
data_path = "/content/drive/MyDrive/zindidata/french1.txt"
data_path2 = "/content/drive/MyDrive/zindidata/fon1.txt"
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().strip().split('\n')
with open(data_path2, 'r', encoding='utf-8') as f:
  lines2 = f.read().strip().split('\n')

lines = [" ".join(re.findall(r"[A-Za-z0-9]+",line)) for line in lines]
lines2 = [" ".join(re.findall(r"[A-Za-z0-9]+",line)) for line in lines2]#[re.sub(r"%s||<|>|%|[a-z]|[A-Z]|_",'',line) for line in lines2]
#print(lines[:5])

# Grouping lines by response pair
pairs = list(zip(lines,lines2))
random.shuffle(pairs)
print(len(pairs))

53134


In [4]:
# Diviser les paires en données d'entrée et de sortie
input_sequences = [pair[0] for pair in pairs]
target_sequences = [pair[1] for pair in pairs]


In [5]:
# Assurez-vous que toutes les séquences ont la même longueur
max_len_input = max(len(seq) for seq in input_sequences)
max_len_target = max(len(seq) for seq in target_sequences)

In [6]:
# Tokenization : convertir les mots en indices

tokenizer_inputs = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer_outputs = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer_inputs.fit_on_texts(input_sequences)
tokenizer_outputs.fit_on_texts(target_sequences)

input_sequences = tokenizer_inputs.texts_to_sequences(input_sequences)
target_sequences = tokenizer_outputs.texts_to_sequences(target_sequences)


In [7]:

# Remarque : assurez-vous d'ajuster vos séquences d'entrée et de sortie à la même longueur maximale
encoder_inputs_padded = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_len_input, padding='post')
decoder_inputs_padded = tf.keras.preprocessing.sequence.pad_sequences(target_sequences, maxlen=max_len_target, padding='post')


In [8]:
# Créez les séquences de sortie décalées d'une position vers la droite
# Cela est nécessaire car le modèle prédit le mot suivant à partir du mot actuel
decoder_outputs_padded = np.zeros_like(decoder_inputs_padded)
decoder_outputs_padded[:, 0:-1] = decoder_inputs_padded[:, 1:]
decoder_outputs_padded[:, -1] = 0  # Remplacer le dernier token par un token de padding


In [12]:

# Création du modèle
embedding_dim = 256
units = 1024

In [13]:
# Encodeur
encoder_inputs = tf.keras.layers.Input(shape=(max_len_input,))
encoder_embedding = tf.keras.layers.Embedding(len(tokenizer_inputs.word_index) + 1, embedding_dim)
encoder_lstm = tf.keras.layers.LSTM(units, return_state=True)

encoder_embedding_output = encoder_embedding(encoder_inputs)
_, state_h, state_c = encoder_lstm(encoder_embedding_output)
encoder_states = [state_h, state_c]

In [14]:

# Décodeur
decoder_inputs = tf.keras.layers.Input(shape=(max_len_target,))
decoder_embedding = tf.keras.layers.Embedding(len(tokenizer_outputs.word_index) + 1, embedding_dim)
decoder_lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)
decoder_dense = tf.keras.layers.Dense(len(tokenizer_outputs.word_index) + 1, activation='softmax')

decoder_embedding_output = decoder_embedding(decoder_inputs)
decoder_lstm_output, _, _ = decoder_lstm(decoder_embedding_output, initial_state=encoder_states)
decoder_outputs = decoder_dense(decoder_lstm_output)

In [17]:
# Modèle complet
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [51]:

# Entrainement du modèle
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#model.fit([encoder_inputs_padded, decoder_inputs_padded], np.expand_dims(decoder_outputs_padded, -1), batch_size=64, epochs=10, validation_split=0.2)



Epoch 1/10
665/665 [==============================] - 371s 550ms/step - loss: 0.1878 - accuracy: 0.9738 - val_loss: 0.1204 - val_accuracy: 0.9782
Epoch 2/10
665/665 [==============================] - 368s 553ms/step - loss: 0.1168 - accuracy: 0.9783 - val_loss: 0.1136 - val_accuracy: 0.9787
Epoch 3/10
665/665 [==============================] - 367s 552ms/step - loss: 0.1107 - accuracy: 0.9787 - val_loss: 0.1089 - val_accuracy: 0.9790
Epoch 4/10
665/665 [==============================] - 368s 553ms/step - loss: 0.1060 - accuracy: 0.9791 - val_loss: 0.1050 - val_accuracy: 0.9793
Epoch 5/10
665/665 [==============================] - 368s 553ms/step - loss: 0.1017 - accuracy: 0.9794 - val_loss: 0.1022 - val_accuracy: 0.9796
Epoch 6/10
665/665 [==============================] - 367s 552ms/step - loss: 0.0976 - accuracy: 0.9798 - val_loss: 0.0992 - val_accuracy: 0.9799
Epoch 7/10
665/665 [==============================] - 369s 556ms/step - loss: 0.0937 - accuracy: 0.9802 - val_loss: 0.0953 -

In [53]:

# Sauvegarde du modèle
model.save('translation_model.h5')

In [15]:
import matplotlib.pyplot as plt

# Entrainement du modèle avec enregistrement de l'historique
#history = model.fit([encoder_inputs_padded, decoder_inputs_padded], np.expand_dims(decoder_outputs_padded, -1), batch_size=64, epochs=10, validation_split=0.2)

# Affichage de la courbe d'apprentissage
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [9]:
# Charger le modèle entraîné
model = tf.keras.models.load_model('translation_model.h5')

# Définir encoder_model et decoder_model
# Placer ici le code pour définir encoder_model et decoder_model

# Fonction pour effectuer une traduction
def translate_sentence(input_sentence):
    # Prétraiter l'entrée
    input_sequence = tokenizer_inputs.texts_to_sequences([input_sentence])
    input_sequence = tf.keras.preprocessing.sequence.pad_sequences(input_sequence, maxlen=max_len_input, padding='post')

    # Encodeur
    encoder_input = input_sequence
    encoder_output, encoder_state_h, encoder_state_c = encoder_model.predict(encoder_input)
    encoder_states = [encoder_state_h, encoder_state_c]

    # Début de la séquence de sortie
    target_sequence = np.zeros((1, 1))
    target_sequence[0, 0] = tokenizer_outputs.word_index['<start>']

    # Traduction de la phrase
    translated_sentence = ''
    stop_condition = False
    while not stop_condition:
        decoder_output, decoder_state_h, decoder_state_c = decoder_model.predict([target_sequence] + encoder_states)

        # Index du mot prédit
        sampled_token_index = np.argmax(decoder_output[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]

        # Arrêt si on atteint la fin de la phrase ou si la longueur maximale est atteinte
        if sampled_word == '<end>' or len(translated_sentence.split()) > max_len_target:
            stop_condition = True
        else:
            translated_sentence += sampled_word + ' '

        # Mise à jour de l'état du décodeur
        encoder_states = [decoder_state_h, decoder_state_c]

        # Mettre à jour la séquence de sortie
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index

    return translated_sentence

# Exemple d'utilisation de la fonction de traduction
input_sentence = 'bonjour'
translated_sentence = translate_sentence(input_sentence)
print('Input Sentence:', input_sentence)
print('Translated Sentence:', translated_sentence)


OSError: No file or directory found at translation_model.h5